In [2]:
# Zelle 1: Setup & Config
from google.cloud import bigquery
import pandas as pd
import logging

# Config
PROJECT_ID = "taxi-bi-project"  
DIM_DATASET = "dimensional"     # Quelle (Star Schema)
AGG_DATASET = "aggregational"       # Ziel (Data Marts)

client = bigquery.Client(project=PROJECT_ID)

# Hilfsfunktion, um Datasets zu finden/erstellen
def create_dataset_if_not_exists(dataset_id):
    full_dataset_id = f"{PROJECT_ID}.{dataset_id}"
    try:
        client.get_dataset(full_dataset_id)
        print(f"✅ Ziel-Dataset gefunden: {full_dataset_id}")
    except:
        print(f"Erstelle neues Dataset: {full_dataset_id} ...")
        # Wir holen uns die Region vom Quell-Dataset, damit alles gleich liegt (EU/US)
        src_ds = client.get_dataset(f"{PROJECT_ID}.{DIM_DATASET}")
        new_ds = bigquery.Dataset(full_dataset_id)
        new_ds.location = src_ds.location
        client.create_dataset(new_ds)
        print(f"✅ Dataset erstellt (Region: {src_ds.location})")

create_dataset_if_not_exists(AGG_DATASET)

✅ Ziel-Dataset gefunden: taxi-bi-project.aggregational


In [14]:
# Zelle 2: Aggregation 1 - Monthly KPIs (Management View)
# Diese Tabelle beantwortet: "Wie entwickeln sich Umsatz und Fahrtenzahlen?"

def create_monthly_kpis():
    print("--- 1. Erstelle Tabelle: agg_monthly_kpis ---")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_monthly_kpis`
    AS
    SELECT
        -- Dimensionen (Woran wollen wir schneiden?)
        d.year,
        d.month,
        d.month_name,
        d.quarter,
        f.source_system,      -- Yellow vs Green vs FHV
        v.vendor_name,        -- Creative Mobile vs Uber/Lyft Bases
        p.payment_description, -- Cash vs Credit
        
        -- Metriken (Hier wird gerechnet!)
        COUNT(f.trip_id) AS total_trips,
        
        -- Summen (Runden auf 2 Nachkommastellen spart Speicher und sieht besser aus)
        ROUND(SUM(f.total_amount), 2) AS total_revenue,
        ROUND(SUM(f.fare_amount), 2) AS total_fare,
        ROUND(SUM(f.tip_amount), 2) AS total_tips,
        
        -- Durchschnitte (KPIs)
        ROUND(AVG(f.total_amount), 2) AS avg_ticket_size,
        ROUND(AVG(f.trip_distance), 2) AS avg_distance_miles,
        ROUND(AVG(f.duration_minutes), 1) AS avg_duration_min

    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    
    -- JOINs zum Star Schema
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON f.pickup_date_key = d.date_key
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_vendor` v ON f.vendor_id = v.vendor_id
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_payment_type` p ON f.payment_type_id = p.payment_type_id

    -- FILTER:
    -- Wir wollen hier nur "echte" Fahrten für die Statistik.
    -- Wir schließen 0$-Fahrten (Geister) aus.
    -- Aber: Deine 3.80$ Fahrt (Short Trip) bleibt drin, weil total_amount > 0!
    WHERE f.total_amount > 0
    
    GROUP BY 1, 2, 3, 4, 5, 6, 7
    ORDER BY year DESC, month DESC, total_revenue DESC
    """
    
    try:
        job = client.query(sql)
        job.result() # Warten auf Fertigstellung
        print("✅ agg_monthly_kpis erfolgreich erstellt.")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_monthly_kpis()

--- 1. Erstelle Tabelle: agg_monthly_kpis ---
✅ agg_monthly_kpis erfolgreich erstellt.


In [15]:
def create_geo_stats_final():
    print("--- 2. Erstelle Tabelle: agg_geo_stats ---")
    
    # Wir nutzen SAFE_CAST und FORMAT_DATE, um sicherzugehen, dass der Key matcht
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_geo_stats` AS
    SELECT
        d.year,
        d.quarter,
        loc.borough AS pickup_borough,
        loc.zone AS pickup_zone,
        IFNULL(loc.service_zone, 'Other') AS service_zone,
        f.source_system,
        
        COUNT(f.trip_id) AS pickup_count,
        ROUND(SUM(f.total_amount), 0) AS total_revenue_generated,
        ROUND(AVG(f.tip_amount), 2) AS avg_tip_here

    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    
    -- JOIN zur Datumstabelle
    -- Wir wandeln das Trip-Datum in das Format YYYYMMDD um (Standard für date_key)
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d 
        ON CAST(FORMAT_DATE('%Y%m%d', DATE(f.pickup_datetime)) AS INT64) = CAST(d.date_key AS INT64)
        
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id

    WHERE f.total_amount > 0 
    GROUP BY 1, 2, 3, 4, 5, 6
    ORDER BY pickup_count DESC
    """
    
    try:
        client.query(sql).result()
        print("✅ agg_geo_stats erfolgreich erstellt!")
    except Exception as e:
        # Falls date_key doch ein DATE-Typ ist, w Fallback:
    
        fallback_sql = sql.replace(
            "CAST(FORMAT_DATE('%Y%m%d', DATE(f.pickup_datetime)) AS INT64) = CAST(d.date_key AS INT64)",
            "DATE(f.pickup_datetime) = d.date_key"
        )
        try:
            client.query(fallback_sql).result()
            print("✅ agg_geo_stats erfolgreich erstellt!")
        except Exception as e_inner:
            print(f"Kritischer Fehler: {e_inner}")

create_geo_stats_final()

--- 2. Erstelle Tabelle: agg_geo_stats ---
✅ agg_geo_stats erfolgreich erstellt!


In [23]:
# Zelle 4: Quality Check (Kurzer Blick auf das Ergebnis)
# Prüfen, ob die Tabellen gefüllt sind und die Geisterfahrten weg sind

def check_aggregation():
    print("\n--- CHECK: Monthly KPIs (Top 5 Rows) ---")
    query_kpi = f"""
    SELECT year, month_name, source_system, total_trips, total_revenue, avg_ticket_size 
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_monthly_kpis` 
    ORDER BY total_trips DESC 
    LIMIT 5
    """
    print(client.query(query_kpi).to_dataframe().to_string(index=False))

    print("\n--- CHECK: Geo Stats (Top 5 Zones) ---")
    query_geo = f"""
    SELECT pickup_borough, pickup_zone, source_system, pickup_count, total_revenue_generated
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_geo_stats`
    WHERE pickup_zone != 'NV' -- Wir ignorieren kurz die Unknowns
    ORDER BY pickup_count DESC
    LIMIT 5
    """
    print(client.query(query_geo).to_dataframe().to_string(index=False))

check_aggregation()


--- CHECK: Monthly KPIs (Top 5 Rows) ---


/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


 year month_name source_system  total_trips  total_revenue  avg_ticket_size
 2010       June        YELLOW      4628917    48508294.19            10.48
 2011       June        YELLOW      4232780    44625418.33            10.54
 2010       June        YELLOW      4225814    43097295.83            10.20
 2011       June        YELLOW      4108120    44230599.26            10.77
 2015       June        YELLOW      4056209    73857571.43            18.21

--- CHECK: Geo Stats (Top 5 Zones) ---
pickup_borough           pickup_zone source_system  pickup_count  total_revenue_generated
     Manhattan Upper East Side South        YELLOW        541769                5080112.0
     Manhattan        Midtown Center        YELLOW        532814                5729221.0
     Manhattan Upper East Side South        YELLOW        530182                5210931.0
     Manhattan Upper East Side South        YELLOW        522302                5185967.0
     Manhattan        Midtown Center        YELLOW    

/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [24]:
def create_time_trends():
    print("--- 3. Erstelle Tabelle: agg_time_trends ---")
    
    sql = f"""
    CREATE OR REPLACE TABLE `taxi-bi-project.dimensional.dim_date` AS
    SELECT
        datum AS date_key,
        EXTRACT(YEAR FROM datum) AS year,
        EXTRACT(MONTH FROM datum) AS month,
        FORMAT_DATE('%B', datum) AS month_name,
        FORMAT_DATE('%A', datum) AS day_name,             -- WICHTIG: Für dein Skript
        EXTRACT(DAYOFWEEK FROM datum) AS day_of_week_num, -- WICHTIG: Für dein Skript
        EXTRACT(QUARTER FROM datum) AS quarter,
        CASE WHEN EXTRACT(DAYOFWEEK FROM datum) IN (1, 7) THEN TRUE ELSE FALSE END AS is_weekend
    FROM UNNEST(GENERATE_DATE_ARRAY('2010-01-01', '2025-12-31')) AS datum;
    """
    try:
        client.query(sql).result()
        print("✅ agg_time_trends erstellt.")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_time_trends()

--- 3. Erstelle Tabelle: agg_time_trends ---
✅ agg_time_trends erstellt.


In [25]:
def create_route_stats():
    print("--- 4. Erstelle Tabelle: agg_route_stats ---")
    
    # Hinweis: Ich habe LocationID zu location_id geändert. 
    # Bitte prüfe, ob die Spalte in deiner dim_location tatsächlich so heißt.
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_route_stats` AS
    SELECT
        d.year,
        d.month, -- Empfehlung: Monat hinzufügen für bessere Zeitreihen
        
        -- VON -> NACH
        pu.Borough AS pickup_borough,
        do.Borough AS dropoff_borough,
        
        COUNT(*) AS trip_count,
        ROUND(AVG(f.total_amount), 2) AS avg_cost,
        -- Falls duration_minutes in Fact_Trips existiert:
        ROUND(AVG(SAFE_DIVIDE(TIMESTAMP_DIFF(f.dropoff_datetime, f.pickup_datetime, SECOND), 60)), 1) AS avg_duration_min
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    -- Join auf deine neue dim_date (oder dim_datetime)
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON DATE(f.pickup_datetime) = d.date_key
    -- Achte hier auf den Spaltennamen: location_id vs LocationID
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` pu ON f.pickup_location_id = pu.location_id
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` do ON f.dropoff_location_id = do.location_id
    
    WHERE f.total_amount > 0 
      AND pu.Borough != 'Unknown' 
      AND do.Borough != 'Unknown'
      
    GROUP BY 1, 2, 3, 4
    ORDER BY trip_count DESC
    """
    try:
        client.query(sql).result()
        print("✅ agg_route_stats erfolgreich erstellt.")
    except Exception as e:
        # Detailliertere Fehlerausgabe
        print(f"❌ Fehler bei der Erstellung von agg_route_stats: {e}")

create_route_stats()

--- 4. Erstelle Tabelle: agg_route_stats ---
✅ agg_route_stats erfolgreich erstellt.


In [26]:
def create_airport_stats():
    print("--- 5. Erstelle Tabelle: agg_airport_trips ---")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_airport_trips` AS
    SELECT
        d.year,
        d.month_name,
        f.source_system,
        
        -- War es eine Fahrt ZUM oder VOM Flughafen?
        CASE 
            WHEN rc.rate_description LIKE '%JFK%' OR rc.rate_description LIKE '%Newark%' THEN 'Airport Rate'
            ELSE 'Standard Rate to Airport Zone'
        END AS trip_category,
        
        COUNT(*) AS total_trips,
        ROUND(AVG(f.total_amount), 2) AS avg_ticket,
        ROUND(AVG(f.tip_amount), 2) AS avg_tip
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON f.pickup_date_key = d.date_key
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_rate_code` rc ON f.rate_code_id = rc.rate_code_id
    
    -- Location Filter: 132=JFK, 138=LaGuardia, 1=Newark
    WHERE (f.pickup_location_id IN (132, 138, 1) OR f.dropoff_location_id IN (132, 138, 1))
      AND f.total_amount > 0
      
    GROUP BY 1, 2, 3, 4
    """
    try:
        client.query(sql).result()
        print("✅ agg_airport_trips erstellt.")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_airport_stats()

--- 5. Erstelle Tabelle: agg_airport_trips ---
✅ agg_airport_trips erstellt.


In [29]:
def create_quality_audit_mart():
    print("--- 8. Erstelle Tabelle: agg_quality_audit ---")
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_quality_audit` AS
    SELECT
        DATE_TRUNC(pickup_datetime, MONTH) as month,
        source_system,
        COUNT(*) as total_trips,
        COUNTIF(trip_distance = 0 AND source_system != 'FHV') as gps_failures,
        COUNTIF(pickup_location_id IN (263, 264)) as unknown_locations,
        COUNTIF(dq_issue_flag = TRUE) as total_issues
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips`
    GROUP BY 1, 2
    """
    client.query(sql).result()
    print("✅ agg_quality_audit erstellt.")
create_quality_audit_mart()

--- 8. Erstelle Tabelle: agg_quality_audit ---
✅ agg_quality_audit erstellt.


In [30]:
def create_shared_ride_stats():
    print("--- 7. Erstelle Tabelle: agg_shared_rides ---")
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_shared_rides` AS
    SELECT
        d.year,
        f.source_system,
        f.sr_flag,
        COUNT(*) AS trip_count,
        ROUND(AVG(f.fare_amount), 2) AS avg_fare -- Nur für Yellow/Green sinnvoll
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON f.pickup_date_key = d.date_key
    GROUP BY 1, 2, 3
    """
    client.query(sql).result()
    print("✅ agg_shared_rides erstellt.")
create_shared_ride_stats()

--- 7. Erstelle Tabelle: agg_shared_rides ---
✅ agg_shared_rides erstellt.


In [22]:
def check_new_marts():
    # Stelle sicher, dass die Variablen passen
    PROJECT_ID = "taxi-bi-project"
    AGG_DATASET = "aggregational"
    
    print("--- 📊 FINAL DATA MART CHECK ---")

    # 1. RUSH HOUR (Wann ist am meisten los?)
    print("\n🕐 TOP 5 ZEITFENSTER (agg_time_trends):")
    # Wir sortieren nach trip_count, um die geschäftigsten Stunden zu sehen
    sql_time = f"""
    SELECT day_name, hour_of_day, source_system, trip_count, avg_fare
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_time_trends`
    ORDER BY trip_count DESC
    LIMIT 5
    """
    try:
        df_time = client.query(sql_time).to_dataframe()
        print(df_time.to_string(index=False))
    except Exception as e:
        print(f"Fehler: {e}")

    # 2. ROUTEN (Wer fährt von wo nach wo?)
    print("\n📍 TOP 5 ROUTEN (agg_route_stats):")
    sql_routes = f"""
    SELECT pickup_borough, dropoff_borough, trip_count, avg_cost
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_route_stats`
    ORDER BY trip_count DESC
    LIMIT 5
    """
    try:
        df_routes = client.query(sql_routes).to_dataframe()
        print(df_routes.to_string(index=False))
    except Exception as e:
        print(f"Fehler: {e}")

    # 3. FLUGHAFEN (Die Cash Cows)
    print("\n✈️ FLUGHAFEN STATS (agg_airport_trips):")
    sql_air = f"""
    SELECT year, trip_category, source_system, total_trips, avg_ticket
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_airport_trips`
    ORDER BY total_trips DESC
    LIMIT 5
    """
    try:
        df_air = client.query(sql_air).to_dataframe()
        print(df_air.to_string(index=False))
    except Exception as e:
        print(f"Fehler: {e}")

check_new_marts()

--- 📊 FINAL DATA MART CHECK ---

🕐 TOP 5 ZEITFENSTER (agg_time_trends):


/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


 day_name  hour_of_day source_system  trip_count  avg_fare
 Thursday           18        YELLOW      429644     29.42
Wednesday           18        YELLOW      423982     28.90
  Tuesday           18        YELLOW      403973     28.57
   Friday           18        YELLOW      397860     28.56
 Thursday           17        YELLOW      396463     31.39

📍 TOP 5 ROUTEN (agg_route_stats):


/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


pickup_borough dropoff_borough  trip_count  avg_cost
     Manhattan       Manhattan    12410566     10.13
     Manhattan       Manhattan    12330416      9.96
     Manhattan       Manhattan    12316988      9.62
     Manhattan       Manhattan    11819999     11.80
     Manhattan       Manhattan    11183007     12.05

✈️ FLUGHAFEN STATS (agg_airport_trips):
 year                 trip_category source_system  total_trips  avg_ticket
 2014 Standard Rate to Airport Zone        YELLOW       614827       41.43
 2013 Standard Rate to Airport Zone        YELLOW       601767       40.22
 2012 Standard Rate to Airport Zone        YELLOW       596237       33.02
 2015 Standard Rate to Airport Zone        YELLOW       587991       43.20
 2011 Standard Rate to Airport Zone        YELLOW       578500       32.69


/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [13]:
# Funktion für 1) Peak Hours – Taxi Demand
def create_agg_peak_hours():
    """
    Erstellt die Tabelle 'agg_peak_hours' im Aggregational Layer.
    
    Logik:
    1. Basis: dimensional.Fact_Trips (Hier sind die Zeitstempel und die Fahrten selbst)
    2. Join: dimensional.dim_location (Um statt LocationIDs echte Borough-Namen zu haben)
    3. Ergebnis: Eine kleine, schnelle Tabelle, die nur noch Stunden und Anzahl enthält.
    """
    
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_peak_hours"
    print(f"🔄 Aktualisiere Peak Hours (mit Monat): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        EXTRACT(MONTH FROM f.pickup_datetime) as month, -- NEU
        EXTRACT(HOUR FROM f.pickup_datetime) as hour,
        
        f.source_system as taxi_type,
        COALESCE(loc.Borough, 'Unknown') as borough,
        
        COUNT(f.trip_id) as trip_count
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
        
    WHERE f.pickup_datetime IS NOT NULL
    
    GROUP BY year, month, hour, taxi_type, borough -- NEU: month im Group By
    """
    
    try:
        client.query(sql).result()
        print(f"✅ Tabelle '{table_id}' erfolgreich aktualisiert!")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_agg_peak_hours()

🔄 Aktualisiere Peak Hours (mit Monat): taxi-bi-project.aggregational.agg_peak_hours ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_peak_hours' erfolgreich aktualisiert!


In [14]:
# Funktion für 2) Fare Distribution – Boxplot Stats
def create_agg_fare_stats():
    """
    Erstellt die Tabelle 'agg_fare_stats' im Aggregational Layer.
    
    Zweck: 
    Bereitstellung von statistischen Daten für Boxplots (Preisverteilung).
    Anstatt Rohdaten zu laden, berechnen wir Quantile (Min, 25%, Median, 75%, Max).
    
    Logik:
    1. Basis: dimensional.Fact_Trips (für fare_amount)
    2. Join: dimensional.dim_location (für Boroughs)
    3. Berechnung: APPROX_QUANTILES teilt die Daten in 100 Teile.
    """
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_fare_stats"
    print(f"🔄 Aktualisiere Fare Stats (mit Monat): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        EXTRACT(MONTH FROM f.pickup_datetime) as month, -- NEU
        f.source_system as taxi_type,
        COALESCE(loc.Borough, 'Unknown') as borough,
        
        -- Quantile müssen jetzt pro Monat berechnet werden
        APPROX_QUANTILES(f.fare_amount, 100)[OFFSET(0)] as min_fare,
        APPROX_QUANTILES(f.fare_amount, 100)[OFFSET(25)] as q1_fare,
        APPROX_QUANTILES(f.fare_amount, 100)[OFFSET(50)] as median_fare,
        APPROX_QUANTILES(f.fare_amount, 100)[OFFSET(75)] as q3_fare,
        APPROX_QUANTILES(f.fare_amount, 100)[OFFSET(95)] as max_fare,
        
        COUNT(*) as trip_count
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
        
    WHERE f.fare_amount > 0 
      AND f.fare_amount < 1000
      AND loc.Borough NOT IN ('Unknown', 'NV')
      
    GROUP BY year, month, taxi_type, borough
    """
    
    try:
        client.query(sql).result()
        print(f"✅ Tabelle '{table_id}' erfolgreich aktualisiert!")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_agg_fare_stats()

🔄 Aktualisiere Fare Stats (mit Monat): taxi-bi-project.aggregational.agg_fare_stats ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_fare_stats' erfolgreich aktualisiert!


In [15]:
def create_agg_tip_stats():
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_tip_stats"
    print(f"🔄 Aktualisiere Tip Stats (mit Monat): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        EXTRACT(MONTH FROM f.pickup_datetime) as month, -- NEU
        f.source_system as taxi_type,
        COALESCE(loc.Borough, 'Unknown') as borough,
        
        SUM(f.tip_amount) as total_tip,
        SUM(f.fare_amount) as total_fare,
        COUNT(*) as card_trips
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_payment_type` pay 
        ON f.payment_type_id = pay.payment_type_id
        
    WHERE pay.payment_description = 'Credit Card' 
      AND f.fare_amount > 0
      AND loc.Borough NOT IN ('Unknown', 'NV')
      
    GROUP BY year, month, taxi_type, borough
    """
    
    try:
        client.query(sql).result()
        print(f"✅ Tabelle '{table_id}' erfolgreich aktualisiert!")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_agg_tip_stats()

🔄 Aktualisiere Tip Stats (mit Monat): taxi-bi-project.aggregational.agg_tip_stats ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_tip_stats' erfolgreich aktualisiert!


In [16]:
# Funktion für 4) Demand Shift over Years
def create_agg_demand_years():
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_demand_years"
    print(f"🔄 Aktualisiere Demand Trends (mit Monat): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        EXTRACT(MONTH FROM f.pickup_datetime) as month, -- NEU
        f.source_system as taxi_type,
        COALESCE(loc.Borough, 'Unknown') as borough,
        COUNT(*) as total_trips
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
    WHERE f.pickup_datetime IS NOT NULL  
    GROUP BY year, month, taxi_type, borough
    """
    try:
        client.query(sql).result()
        print(f"✅ Tabelle '{table_id}' erfolgreich aktualisiert!")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_agg_demand_years()

🔄 Aktualisiere Demand Trends (mit Monat): taxi-bi-project.aggregational.agg_demand_years ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_demand_years' erfolgreich aktualisiert!


In [17]:
def create_agg_weekly_patterns():
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_weekly_patterns"
    print(f"🔄 Aktualisiere Weekly Patterns (mit Monat): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        EXTRACT(MONTH FROM f.pickup_datetime) as month, -- NEU
        f.source_system as taxi_type,
        COALESCE(loc.Borough, 'Unknown') as borough,
        d.day_name,
        d.day_of_week_num as day_of_week,
        EXTRACT(HOUR FROM f.pickup_datetime) as hour,
        COUNT(*) as trip_count
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
    INNER JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d
        ON DATE(f.pickup_datetime) = d.date_key
        
    WHERE f.pickup_datetime IS NOT NULL
    GROUP BY year, month, taxi_type, borough, d.day_name, d.day_of_week_num, hour
    """
    try:
        client.query(sql).result()
        print(f"✅ Tabelle '{table_id}' erfolgreich aktualisiert!")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_agg_weekly_patterns()

🔄 Aktualisiere Weekly Patterns (mit Monat): taxi-bi-project.aggregational.agg_weekly_patterns ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_weekly_patterns' erfolgreich aktualisiert!


In [20]:
def create_agg_fare_dist():
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_fare_dist"
    print(f"🔄 Aktualisiere Fare Dist (mit Monat): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        EXTRACT(MONTH FROM f.pickup_datetime) as month, -- NEU
        f.source_system as taxi_type,
        COALESCE(loc.Borough, 'Unknown') as borough,
        
        ROUND(f.trip_distance * 5) / 5 as dist_bin,
        ROUND(f.fare_amount, 0) as fare_bin,
        
        COUNT(*) as trip_count
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
        
    WHERE f.pickup_datetime IS NOT NULL
      AND f.trip_distance > 0 AND f.trip_distance < 100
      AND f.fare_amount > 0 AND f.fare_amount < 500
      
    GROUP BY year, month, taxi_type, borough, dist_bin, fare_bin
    """
    client.query(sql).result()
    print(f"✅ Tabelle '{table_id}' aktualisiert.")

create_agg_fare_dist()

🔄 Aktualisiere Fare Dist (mit Monat): taxi-bi-project.aggregational.agg_fare_dist ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_fare_dist' aktualisiert.


In [21]:
def create_agg_flows():
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_borough_flows"
    print(f"🔄 Aktualisiere Borough Flows (mit Monat): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        EXTRACT(MONTH FROM f.pickup_datetime) as month, -- NEU
        f.source_system as taxi_type,
        
        COALESCE(loc_pu.Borough, 'Unknown') as pickup_borough,
        COALESCE(loc_do.Borough, 'Unknown') as dropoff_borough,
        
        COUNT(*) as trips
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc_pu 
        ON f.pickup_location_id = loc_pu.location_id
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc_do 
        ON f.dropoff_location_id = loc_do.location_id
        
    WHERE f.pickup_datetime IS NOT NULL
      AND loc_pu.Borough IS NOT NULL AND loc_pu.Borough != 'Unknown'
      AND loc_do.Borough IS NOT NULL AND loc_do.Borough != 'Unknown'
      
    GROUP BY year, month, taxi_type, pickup_borough, dropoff_borough
    """
    client.query(sql).result()
    print(f"✅ Tabelle '{table_id}' aktualisiert.")

create_agg_flows()

🔄 Aktualisiere Borough Flows (mit Monat): taxi-bi-project.aggregational.agg_borough_flows ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_borough_flows' aktualisiert.


In [22]:
def create_agg_revenue_efficiency():
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_revenue_efficiency"
    print(f"🔄 Aktualisiere Efficiency (mit Monat): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    WITH raw_calc AS (
        SELECT
            EXTRACT(YEAR FROM f.pickup_datetime) as year,
            EXTRACT(MONTH FROM f.pickup_datetime) as month, -- NEU
            f.source_system as taxi_type,
            COALESCE(loc.Borough, 'Unknown') as borough,
            TIMESTAMP_DIFF(f.dropoff_datetime, f.pickup_datetime, MINUTE) as duration_min,
            f.fare_amount
        FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
        LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc
            ON f.pickup_location_id = loc.location_id
        WHERE f.pickup_datetime IS NOT NULL
          AND f.fare_amount > 0
          AND TIMESTAMP_DIFF(f.dropoff_datetime, f.pickup_datetime, MINUTE) BETWEEN 1 AND 180
    ),
    categorized AS (
        SELECT *,
            CASE
                WHEN duration_min < 10 THEN '1. Kurzstrecke (< 10 min)'
                WHEN duration_min < 20 THEN '2. Mittel (10 - 20 min)'
                WHEN duration_min < 45 THEN '3. Lang (20 - 45 min)'
                ELSE '4. Sehr Lang (> 45 min)'
            END as trip_category,
            SAFE_DIVIDE(fare_amount, duration_min) as fare_per_min
        FROM raw_calc
    )
    SELECT
        year, month, taxi_type, borough, trip_category,
        COUNT(*) as total_trips,
        APPROX_QUANTILES(fare_per_min, 4) as quantiles
    FROM categorized
    GROUP BY year, month, taxi_type, borough, trip_category
    """
    client.query(sql).result()
    print(f"✅ Tabelle '{table_id}' aktualisiert.")

create_agg_revenue_efficiency()

🔄 Aktualisiere Efficiency (mit Monat): taxi-bi-project.aggregational.agg_revenue_efficiency ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_revenue_efficiency' aktualisiert.


In [21]:
def create_agg_location_map():
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_location_map"
    print(f"🔄 Erstelle Location Map NEU (mit avg_amount): {table_id} ...")
    
    # Wir erstellen die Tabelle komplett neu
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        EXTRACT(MONTH FROM f.pickup_datetime) as month,
        f.source_system as taxi_type,
        
        f.pickup_location_id as location_id,
        loc.Zone as zone,
        loc.Borough as borough,
        loc.geojson_str, 
        
        COUNT(*) as trip_count,
        
        -- HIER IST DIE FEHLENDE SPALTE:
        AVG(f.total_amount) as avg_amount
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
        
    WHERE f.pickup_datetime IS NOT NULL
      AND loc.Borough != 'Unknown'
      AND loc.geojson_str IS NOT NULL 
      
    GROUP BY 1, 2, 3, 4, 5, 6, 7
    """
    
    try:
        client.query(sql).result()
        print(f"✅ Tabelle '{table_id}' erfolgreich aktualisiert (Spalte avg_amount hinzugefügt)!")
    except Exception as e:
        print(f"❌ SQL FEHLER: {e}")

create_agg_location_map()

--- Erstelle Tabelle: agg_location_map (Optimiert) ---
✅ agg_location_map erfolgreich erstellt.


In [25]:
def create_agg_airport_connectivity():
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_airport_connectivity"
    print(f"🔄 Korrigiere Airport Tabelle (total_trips Spalte): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    WITH raw_trips AS (
        SELECT
            EXTRACT(YEAR FROM f.pickup_datetime) as year,
            EXTRACT(MONTH FROM f.pickup_datetime) as month,
            f.source_system as taxi_type,
            f.total_amount,
            f.fare_amount,
            f.tip_amount,
            f.payment_type_id,
            
            loc_pu.Zone as pu_zone,
            loc_pu.Borough as pu_borough,
            loc_do.Zone as do_zone,
            loc_do.Borough as do_borough,
            f.rate_code_id
        FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
        LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc_pu ON f.pickup_location_id = loc_pu.location_id
        LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc_do ON f.dropoff_location_id = loc_do.location_id
        WHERE f.pickup_datetime IS NOT NULL 
          AND f.total_amount > 0
    )
    SELECT
        year,
        month,
        taxi_type,
        
        -- Dimensions
        CASE 
            WHEN rate_code_id = 2 OR pu_zone LIKE '%JFK%' OR do_zone LIKE '%JFK%' THEN 'JFK'
            WHEN rate_code_id = 3 OR pu_zone LIKE '%Newark%' OR do_zone LIKE '%Newark%' THEN 'EWR'
            WHEN pu_zone LIKE '%LaGuardia%' OR do_zone LIKE '%LaGuardia%' THEN 'LGA'
            ELSE 'Other'
        END as airport,

        CASE 
            WHEN rate_code_id = 2 OR rate_code_id = 3 OR pu_zone LIKE '%Airport%' THEN 'From Airport'
            ELSE 'To Airport'
        END as direction,

        CASE 
            WHEN rate_code_id = 2 OR rate_code_id = 3 OR pu_zone LIKE '%Airport%' THEN do_borough 
            ELSE pu_borough 
        END as connected_borough,

        -- Metrics
        SUM(total_amount) as total_revenue,
        
        -- HIER WAR DER FEHLER: Wir nennen es jetzt 'total_trips', wie in Python erwartet
        COUNT(*) as total_trips,
        
        SUM(fare_amount) as total_fare_all,
        SUM(CASE WHEN payment_type_id = 1 THEN tip_amount ELSE 0 END) as total_tip,
        SUM(CASE WHEN payment_type_id = 1 THEN fare_amount ELSE 0 END) as total_fare_card

    FROM raw_trips
    WHERE 
       (rate_code_id IN (2,3) 
        OR pu_zone LIKE '%Airport%' 
        OR do_zone LIKE '%Airport%')
    
    GROUP BY 1, 2, 3, 4, 5, 6
    """
    
    try:
        client.query(sql).result()
        print(f"✅ Tabelle '{table_id}' erfolgreich korrigiert!")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_agg_airport_connectivity()

🔄 Korrigiere Airport Tabelle (total_trips Spalte): taxi-bi-project.aggregational.agg_airport_connectivity ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_airport_connectivity' erfolgreich korrigiert!


In [18]:
# Zelle: Aggregation 10 - Tip Deep Dive (Update: Feinere Granularität > 25%)

def create_tip_deepdive_tables():
    # 1. Distribution
    table_dist = f"{PROJECT_ID}.{AGG_DATASET}.agg_tip_distribution"
    print(f"🔄 Aktualisiere Tip Distribution (mit Monat): {table_dist} ...")
    
    sql_dist = f"""
    CREATE OR REPLACE TABLE `{table_dist}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        EXTRACT(MONTH FROM f.pickup_datetime) as month, -- NEU
        f.source_system as taxi_type,
        COALESCE(loc.Borough, 'Unknown') as borough,
        
        CASE 
            WHEN f.tip_amount = 0 THEN '0% (No Tip)'
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) < 0.10 THEN '0 - 10%'
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) BETWEEN 0.10 AND 0.149 THEN '10 - 15%'
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) BETWEEN 0.149 AND 0.199 THEN '15 - 20%'
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) BETWEEN 0.199 AND 0.249 THEN '20 - 25%'
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) BETWEEN 0.249 AND 0.299 THEN '25 - 30%'
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) BETWEEN 0.299 AND 0.349 THEN '30 - 35%'
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) >= 0.349 THEN '> 35%'
            ELSE 'Unknown'
        END as tip_bin,
        
        CASE 
            WHEN f.tip_amount = 0 THEN 1
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) < 0.10 THEN 2
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) BETWEEN 0.10 AND 0.149 THEN 3
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) BETWEEN 0.149 AND 0.199 THEN 4
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) BETWEEN 0.199 AND 0.249 THEN 5
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) BETWEEN 0.249 AND 0.299 THEN 6
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) BETWEEN 0.299 AND 0.349 THEN 7
            WHEN SAFE_DIVIDE(f.tip_amount, f.fare_amount) >= 0.349 THEN 8
            ELSE 9
        END as bin_order,

        COUNT(*) as trip_count

    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
        
    WHERE f.payment_type_id = 1 AND f.fare_amount > 0 AND f.total_amount > 0
    GROUP BY year, month, taxi_type, borough, tip_bin, bin_order
    """
    
    # 2. Zone Ranking
    table_zones = f"{PROJECT_ID}.{AGG_DATASET}.agg_tip_zone_ranking"
    print(f"🔄 Aktualisiere Tip Zone Ranking (mit Monat): {table_zones} ...")
    
    sql_zones = f"""
    CREATE OR REPLACE TABLE `{table_zones}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        EXTRACT(MONTH FROM f.pickup_datetime) as month, -- NEU
        f.source_system as taxi_type,
        loc.Borough as borough,
        loc.Zone as zone,
        
        COUNT(*) as trips,
        ROUND(SAFE_DIVIDE(SUM(f.tip_amount), SUM(f.fare_amount)) * 100, 1) as avg_tip_pct

    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
        
    WHERE f.payment_type_id = 1 AND f.fare_amount > 0
    GROUP BY year, month, taxi_type, borough, zone
    HAVING trips > 50 
    """

    try:
        client.query(sql_dist).result()
        client.query(sql_zones).result()
        print("✅ Tip-Tabellen erfolgreich aktualisiert!")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_tip_deepdive_tables()

🔄 Aktualisiere Tip Distribution (mit Monat): taxi-bi-project.aggregational.agg_tip_distribution ...
🔄 Aktualisiere Tip Zone Ranking (mit Monat): taxi-bi-project.aggregational.agg_tip_zone_ranking ...
✅ Tip-Tabellen erfolgreich aktualisiert!


In [11]:
def create_agg_seasonality_borough():
    # Wir überschreiben die Tabelle - diesmal REIN auf Basis von Trip-Counts (Nachfrage)
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_seasonality_borough"
    print(f"📈 Erstelle Tabelle: {table_id} (Alle Trips, keine Umsatz-Filter) ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        EXTRACT(MONTH FROM f.pickup_datetime) as month,
        FORMAT_DATE('%B', f.pickup_datetime) as month_name,
        f.source_system as taxi_type,
        COALESCE(loc.Borough, 'Unknown') as borough,
        
        -- WICHTIG: Einfach nur zählen. Das ist die Nachfrage.
        COUNT(*) as total_trips
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
        
    WHERE f.pickup_datetime IS NOT NULL
      -- KEIN Filter auf total_amount > 0 mehr! 
      -- Damit sind auch FHV-Fahrten ohne Preisinfo enthalten.
      
    GROUP BY 1, 2, 3, 4, 5
    """
    try:
        client.query(sql).result()
        print(f"✅ Tabelle '{table_id}' erfolgreich aktualisiert!")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_agg_seasonality_borough()

📈 Erstelle Tabelle: taxi-bi-project.aggregational.agg_seasonality_borough (Alle Trips, keine Umsatz-Filter) ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_seasonality_borough' erfolgreich aktualisiert!


In [19]:
def create_agg_route_revenues():
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_route_revenues"
    print(f"🔄 Aktualisiere Route Revenues (mit Monat): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        EXTRACT(MONTH FROM f.pickup_datetime) as month, -- NEU
        f.source_system as taxi_type,
        COALESCE(loc_pu.Borough, 'Unknown') as pickup_borough,
        COALESCE(loc_do.Borough, 'Unknown') as dropoff_borough,
        
        COUNT(*) as total_trips,
        SUM(f.total_amount) as total_revenue,
        AVG(f.total_amount) as avg_fare,
        AVG(f.trip_distance) as avg_distance
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc_pu 
        ON f.pickup_location_id = loc_pu.location_id
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc_do 
        ON f.dropoff_location_id = loc_do.location_id
        
    WHERE f.pickup_datetime IS NOT NULL 
      AND f.total_amount > 0
      AND loc_pu.Borough != 'Unknown' 
      AND loc_do.Borough != 'Unknown'
      
    GROUP BY year, month, taxi_type, pickup_borough, dropoff_borough
    """
    try:
        client.query(sql).result()
        print(f"✅ Tabelle '{table_id}' erfolgreich aktualisiert!")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_agg_route_revenues()

🔄 Aktualisiere Route Revenues (mit Monat): taxi-bi-project.aggregational.agg_route_revenues ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_route_revenues' erfolgreich aktualisiert!
